# Задание 2. Матричные игры

## Импорт библиотеки для линейного программирования

In [1]:
from pulp import *

# Добавление фукнции для красивой печати матриц

In [2]:
def printA(a):
    for row in a:
        for col in row:
            print("{:8.3f}".format(col), end=" ")
        print("")

## Задание исходной матрицы

In [3]:
A = [
    [5, -6, 3, 0, 12, 4],
    [3, 5, 12, 10, 6, 8],
    [10, 0, -5, -2, -4, 6],
    [8, 6, -4, -2, 14, 5],
    [4, 3, -5, 10, 1, 4]
]

printA(A)

   5.000   -6.000    3.000    0.000   12.000    4.000 
   3.000    5.000   12.000   10.000    6.000    8.000 
  10.000    0.000   -5.000   -2.000   -4.000    6.000 
   8.000    6.000   -4.000   -2.000   14.000    5.000 
   4.000    3.000   -5.000   10.000    1.000    4.000 


## Поиск наименьшего отрицательного числа

In [4]:
min_value = min([min(row) for row in A])
min_value = 0 if min_value > 0 else min_value
min_value

-6

## Рассчет матрицы A', где к элементам прибавлено абсолютное значение min_value + 1

In [5]:
A_1 = []
for i in range(len(A)):
    A_1.append([])
    for j in range(len(A[i])):
        A_1[i].append(A[i][j] + abs(min_value) + 1)
printA(A_1)

  12.000    1.000   10.000    7.000   19.000   11.000 
  10.000   12.000   19.000   17.000   13.000   15.000 
  17.000    7.000    2.000    5.000    3.000   13.000 
  15.000   13.000    3.000    5.000   21.000   12.000 
  11.000   10.000    2.000   17.000    8.000   11.000 


## Поиск значений t

In [6]:
t = [pulp.LpVariable(f"t{i}", lowBound=0) for i in range(len(A_1))]
c = [1 for i in range(len(t))]
p1 = pulp.LpProblem('Задача P1', pulp.const.LpMinimize)  # Создание задачи для минимизации
p1 += sum([t[i] for i in range(len(A_1))]), "целевая фукнция"  # задание t1 + t2 + ... -> min
for i in range(len(A_1[0])):
    p1 += sum([t[j] * A_1[j][i] for j in range(len(A_1))]) >= 1, f"{i}" # Задания суммы t1 * A'[1][1] + ... >= 1
p1.solve()

t_ = [variable.varValue for variable in p1.variables()]
t_

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vladimirusakov/PycharmProjects/Game-theory-problem-solving/venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nd/3d2_qjnx1mb3z8g1yd38kdd00000gn/T/352cc969bd73401a9a29b6a99a38a844-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nd/3d2_qjnx1mb3z8g1yd38kdd00000gn/T/352cc969bd73401a9a29b6a99a38a844-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 47 RHS
At line 54 BOUNDS
At line 55 ENDATA
Problem MODEL has 6 rows, 5 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 5 (0) columns and 30 (0) elements
Perturbing problem by 0.001% of 3.79057 - largest nonzero change 4.3825649e-05 ( 0.0011561757%) - largest zero change 0
0  Obj 0 Primal inf 0.36148683 (6)
5  Obj 0.082449689
Optimal - objective value 0.08244898
O

/Users/vladimirusakov/PycharmProjects/Game-theory-problem-solving/venv/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[0.0, 0.047346939, 0.0032653061, 0.030204082, 0.0016326531]

## Поиск v

In [7]:
v = 1.0 / sum([t_[i] * c[i] for i in range(len(t_))])
v

12.12871278182286

## Поиск значений x

In [8]:
x = [t_[i] * v for i in range(len(t_))]
x

[0.0,
 0.5742574242294872,
 0.03960395983163416,
 0.3663366354166258,
 0.019801980522252717]

## Получение транспонированной матрицы от A'

In [9]:
A_2 = [[A_1[i][j]  for i in range(len(A))] for j in range(len(A[0]))]
printA(A_2)

  12.000   10.000   17.000   15.000   11.000 
   1.000   12.000    7.000   13.000   10.000 
  10.000   19.000    2.000    3.000    2.000 
   7.000   17.000    5.000    5.000   17.000 
  19.000   13.000    3.000   21.000    8.000 
  11.000   15.000   13.000   12.000   11.000 


## Поиск значений u

In [10]:
u = [pulp.LpVariable(f"u{i}", lowBound=0) for i in range(len(A_2))]
c = [1 for i in range(len(u))]
p2 = pulp.LpProblem('Задача P2', pulp.const.LpMaximize) # Создание задачи для максимизации
p2 += sum([u[i] for i in range(len(A_2))]), "целевая фукнция" # задание u1 + u2 + ... -> max
for i in range(len(A_2[0])):
    p2 += sum([u[j] * A_2[j][i] for j in range(len(A_2))]) <= 1, f"{i}" # Задания суммы u1 * A''[1][1] + ... <= 1
p2.solve()

u_ = [variable.varValue for variable in p2.variables()]
u_

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vladimirusakov/PycharmProjects/Game-theory-problem-solving/venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nd/3d2_qjnx1mb3z8g1yd38kdd00000gn/T/a2eb04f9fa4f49578b09184acbece4ff-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/nd/3d2_qjnx1mb3z8g1yd38kdd00000gn/T/a2eb04f9fa4f49578b09184acbece4ff-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 47 RHS
At line 53 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 6 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 6 (0) columns and 30 (0) elements
Perturbing problem by 0.001% of 5.4848276 - largest nonzero change 4.479644e-05 ( 0.0012267052%) - largest zero change 0
0  Obj -0 Dual inf 23.728858 (6)
0  Obj -0 Dual inf 23.729095 (6)
5  Obj 0.08244898
Opt

[0.04664723, 0.015393586, 0.00093294461, 0.019475219, 0.0, 0.0]

## Поиск v

In [11]:
v = 1.0 / sum([u_[i] * c[i] for i in range(len(u_))])
v

12.128712868615208

## Поиск значений y

In [12]:
y = [u_[i] * v for i in range(len(u_))]
y

[0.5657708587862533,
 0.18670438461233493,
 0.011315417297012196,
 0.23620933930439938,
 0.0,
 0.0]